## COMP as Compilable
### Evaluation of Tool Performance for SmartBugs curated
Notebook to study the patches generated by APR tools on smart contracts.
In here we take into consideration if the patch generated by the tool is actually **COMPILABLE**.

#### Elysium and SmartShield have not been added to the data yet 

The data regarding diff analysis is on
'../results/smartbugs/TOOL/compilation_results_0.4.24.csv'

By convention, the result of experiments are allocated on:
'../results/smartbugs/

Details of the original dataset are allocated on:
'../results/smartbugs/contracts.csv'

The results of this notebook will be allocated on:
'../results/smartbugs/tool_patch_ability_COMP_per_contract.csv'

0 is compilable

In [1]:
import pandas as pd
#Example of the input data
tool_file='../results/smartbugs/TIPS/compilation_results_0.4.24.csv'
df=pd.read_csv(tool_file)

In [2]:
#df[df['original'].str.contains("0x3a0e9acd953ffc0dd18d63603488846a6b8b2b01") ].values.tolist()
df.head()

,file,compilation exit code (0: success; 1: failure)
0,unchecked_low_level_calls/0xf70d589d76eebdd7c1...,0
1,unchecked_low_level_calls/0xf70d589d76eebdd7c1...,0
2,unchecked_low_level_calls/0xe894d54dca59cb53fe...,0
3,unchecked_low_level_calls/0xe894d54dca59cb53fe...,0
4,unchecked_low_level_calls/0xb620cee6b52f96f3c6...,0


In [3]:
import os

#collecting contracts name
dataset='smartbugs'
results_path='../results'
contracts_path=os.path.join(results_path,dataset,'contracts.csv')
storage_path=os.path.join(results_path,dataset)

import pandas as pd
contracts_df = pd.read_csv(contracts_path)


In [4]:
contracts_df.head()

,Name,Category,Occurrences,Lines
0,FibonacciBalance.sol,access_control,2,"31,38"
1,arbitrary_location_write_simple.sol,access_control,1,27
2,incorrect_constructor_name1.sol,access_control,1,20
3,incorrect_constructor_name2.sol,access_control,1,18
4,incorrect_constructor_name3.sol,access_control,1,17


In [5]:
#contracts_df.sort_values(by='Name', ascending=True).head(10)

In [6]:
tools = [
    #'Elysium', #bytecode
    'sGuard',  
    'sGuardPlus',
    'SmartFix', 
    #'SmartShield', #bytecde
    "SolGPT" ,
    'TIPS', 
]

In [7]:
def generate_result_dic():
    result_dic={}
    index=0
    for contract , category in contracts_df[["Name","Category"]].values.tolist():
        index_str=str(index)
        result_dic[index_str]={}
        result_dic[index_str]["Contract"]= contract
        result_dic[index_str]["Category"]=category
        for tool in tools:
            result_dic[index_str][tool]=None
        index+=1
    return result_dic
        
initialize_result_dic=generate_result_dic()
    

In [8]:
import re
import os
def get_contract(input_string):
    match = re.search(r'[^/]+$', input_string)
    if match:
        return match.group()
    else:
        print('fail on get_contract')
def get_contract_TIPS(input_string):
    match = re.search(r'/([^/]+)/[^/]+$', input_string)
    if match:
        return match.group(1)
    else:
        print('fail on get_contract_TIPS')
    

def get_category(text):
    match = re.search(r'^([^/]+)', text)
    if match:
        return match.group(1)
    else:
        print('fail on get_category')
        return ''

def read_comp_csv(tool, result_df) :
    read=0
    tool_file='../results/smartbugs/'+tool+'/compilation_results_0.4.24.csv'
    if os.path.exists(tool_file):
        df=pd.read_csv(tool_file)#, skiprows=1)
        values=df.values.tolist()
        for line in values:
            category= get_category(line[0])
            contract = get_contract(line[0])
            value= line[-1]
            if value == 0:
                value = True
            else :
                value = False
            #if tool in ['SmartShield', 'Elysium']:  # We do not consider compilation on binary tools
            #    contract=contract.replace(".rt.hex", ".sol")
            #case tools with multiple patches
            if tool in ["TIPS", "SolGPT"]:
                if tool == "TIPS":
                    contract=get_contract_TIPS(line[0])+'.sol'
                if tool == "SolGPT" and "round" in line[0]:
                    contract=get_contract_TIPS(line[0])+'.sol'
                if result_df[(result_df['Contract'] == contract) & (result_df['Category'] == category)][tool].values.tolist() != [True]:
                    result_df.loc[(result_df['Contract'] == contract) & (result_df['Category'] == category) , tool] = value
            else:
                result_df.loc[(result_df['Contract'] == contract) & (result_df['Category'] == category) , tool] = value
    return result_df
        
def feed_result_dic(initialize_dic, tool_list):
    for tool in tools:
        initialize_dic=read_comp_csv(tool,initialize_dic)
    return initialize_dic

def generate_df():
    initialize_result_dic=generate_result_dic()
    #return initialize_result_dic
    in_df=pd.DataFrame.from_dict(initialize_result_dic, orient='index',columns=['Contract','Category']+tools)
    return feed_result_dic(in_df,tools)

result_dic=generate_df()

In [9]:
result_dic[result_dic ==True].count()

Contract        0
Category        0
sGuard        108
sGuardPlus     81
SmartFix       86
SolGPT        138
TIPS          138
dtype: int64

In [10]:
result_dic[result_dic==False].count()

Contract      0
Category      0
sGuard        1
sGuardPlus    0
SmartFix      0
SolGPT        1
TIPS          2
dtype: int64

In [11]:
#CASE DUPLICATE CONTRACT
result_dic[result_dic["Contract"]== "0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol"]

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS
59,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,reentrancy,True,None,True,True,True
106,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,unchecked_low_level_calls,True,None,True,True,True


In [15]:
result_dic[result_dic["SolGPT"]== False]

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS
16,wallet_03_wrong_constructor.sol,access_control,True,True,True,False,True


In [13]:
#Sanity Check with GENERATION NUMBERS

#If the element has either True or False it means the tool generated a patch, and therefore it needs to match the number from GEN
df=result_dic

#assert len(df[(df['Elysium'] == True) | (df['Elysium'] == False)].values.tolist()) == 126
assert len(df[(df['sGuard'] == True) | (df['sGuard'] == False)].values.tolist()) == 109
assert len(df[(df['sGuardPlus'] == True) | (df['sGuardPlus'] == False)].values.tolist()) == 81
assert len(df[(df['SmartFix'] == True) | (df['SmartFix'] == False)].values.tolist()) == 86
#assert len(df[(df['SmartShield'] == True) | (df['SmartShield'] == False)].values.tolist()) == 134
assert len(df[(df['SolGPT'] == True) | (df['SolGPT'] == False)].values.tolist()) == 139
assert len(df[(df['TIPS'] == True) | (df['TIPS'] == False)].values.tolist()) == 140


In [14]:
#Sanity Check
#Compare with the number on the repository markdown - Not applicable for SolGPT and TIPS (multiple patches)
assert len(df[df['sGuard']==True].values.tolist()) == 108
assert len(df[df['sGuardPlus']==True].values.tolist()) == 81
assert len(df[df['SmartFix']==True].values.tolist()) == 86 
# for multiple patches tool, we assume true if at least one patch compiles
assert len(df[df['SolGPT'] == True].values.tolist()) == 138
assert len(df[df['TIPS']==True].values.tolist()) ==  138 

In [422]:
df=generate_df()

# Columns to consider
columns_to_consider = tools # [ 'sGuard', 'sGuardPlus',  "SmartFix","SolGPT",'TIPS'] #'Elysium', 'SmartShield',

# Summing up the number of True values in each row
df['PatchCount'] = df[columns_to_consider].sum(axis=1)

# Sorting the DataFrame based on the PatchCount column in ascending order
df_sorted = df.sort_values(by='PatchCount', ascending=True)

file_name='tool_patch_ability_COMP_per_contract.csv'
df_sorted.to_csv(os.path.join(storage_path,file_name), index=False)  

## DF for all contracts
- None: means the tool did not generate a patch
- True: means the tool generated a patch and compiles
- False: means the tool did generate a patch but does not compile

In [423]:
df_sorted.head(6)

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
8,parity_wallet_bug_1.sol,access_control,None,None,None,None,None,0
90,timed_crowdsale.sol,time_manipulation,None,None,None,None,True,1
52,name_registrar.sol,other,None,None,None,True,True,2
33,blackjack.sol,bad_randomness,None,None,None,True,True,2
114,0x89c1b3807d4c67df034fffb62f3509561218d30b.sol,unchecked_low_level_calls,True,None,None,True,None,2
130,0xe09b1ab8111c2729a76f16de96bc86a7af837928.sol,unchecked_low_level_calls,None,None,True,True,False,2


## Generate df per category

In [428]:
grouped_df = df_sorted.groupby('Category').sum()

# Extract only the relevant columns for the patch counts
patch_counts = grouped_df[tools] 
patch_counts = patch_counts.replace({True: 1})
# Aggregate the count of unique files per category
files_per_category = contracts_df.groupby('Category')['Name'].nunique().reset_index()
files_per_category.columns = ['Category', 'Total .sol Files']
files_per_category=files_per_category.set_index('Category')

# Merge the total files per category with the main dataframe
per_category_df = patch_counts.join( files_per_category)

total_sum = per_category_df.sum().to_frame().T
total_sum.index = ['Total # contracts with a compilable patch']

# Append the total sum as the last row of the dataframe
final = pd.concat([per_category_df, total_sum])
final=final.reset_index().rename(columns={'index': 'Category'})

#we store it
file_name='tool_patch_ability_COMP_per_category.csv'

final.to_csv(os.path.join(storage_path,file_name), index=False)  

/tmp/ipykernel_23721/2138804270.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  patch_counts = patch_counts.replace({True: 1})


In [427]:
final

,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,Total .sol Files
0,access_control,13,8,13,16,17,18
1,arithmetic,2,11,13,14,15,15
2,bad_randomness,4,1,5,8,7,8
3,denial_of_service,2,2,2,6,6,6
4,front_running,3,1,3,4,4,4
5,other,2,0,2,3,3,3
6,reentrancy,31,28,28,31,30,31
7,short_addresses,0,0,0,1,1,1
8,time_manipulation,4,2,3,4,5,5
9,unchecked_low_level_calls,47,28,17,51,50,52


## Comparison per category
The following DataFrames are grouped by category.

What to look in here:

- What are the contracts with the lowest Patch Count
- How many tools overlaps
- How good each tool is on the category

In [406]:
df_sorted[df_sorted['Category']=='access_control']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
8,parity_wallet_bug_1.sol,access_control,None,None,None,None,None,0
14,unprotected0.sol,access_control,None,None,None,True,True,2
6,multiowned_vulnerable.sol,access_control,None,None,None,True,True,2
9,parity_wallet_bug_2.sol,access_control,None,None,True,True,True,3
11,proxy.sol,access_control,True,None,None,True,True,3
0,FibonacciBalance.sol,access_control,True,None,True,True,True,4
16,wallet_03_wrong_constructor.sol,access_control,True,True,True,False,True,4
1,arbitrary_location_write_simple.sol,access_control,True,None,True,True,True,4
2,incorrect_constructor_name1.sol,access_control,True,None,True,True,True,4
3,incorrect_constructor_name2.sol,access_control,True,None,True,True,True,4


In [407]:
df_sorted[df_sorted['Category']=='arithmetic']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
29,overflow_single_tx.sol,arithmetic,None,True,True,None,True,3
25,integer_overflow_mul.sol,arithmetic,None,True,None,True,True,3
26,integer_overflow_multitx_multifunc_feasible.sol,arithmetic,None,None,True,True,True,3
19,insecure_transfer.sol,arithmetic,None,True,None,True,True,3
20,integer_overflow_1.sol,arithmetic,None,None,True,True,True,3
22,integer_overflow_benign_1.sol,arithmetic,None,None,True,True,True,3
18,BECToken.sol,arithmetic,None,None,True,True,True,3
24,integer_overflow_minimal.sol,arithmetic,None,True,True,True,True,4
31,token.sol,arithmetic,None,True,True,True,True,4
28,overflow_simple_add.sol,arithmetic,None,True,True,True,True,4


In [408]:
df_sorted[df_sorted['Category']=='bad_randomness']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
33,blackjack.sol,bad_randomness,None,None,None,True,True,2
40,smart_billions.sol,bad_randomness,None,None,True,True,None,2
39,random_number_generator.sol,bad_randomness,None,None,None,True,True,2
34,etheraffle.sol,bad_randomness,None,None,True,True,True,3
38,old_blockhash.sol,bad_randomness,True,None,True,True,True,4
37,lucky_doubler.sol,bad_randomness,True,None,True,True,True,4
36,lottery.sol,bad_randomness,True,True,None,True,True,4
35,guess_the_random_number.sol,bad_randomness,True,None,True,True,True,4


In [409]:
df_sorted[df_sorted['Category']=='denial_of_service']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
42,dos_address.sol,denial_of_service,None,None,None,True,True,2
43,dos_number.sol,denial_of_service,None,None,None,True,True,2
44,dos_simple.sol,denial_of_service,None,True,None,True,True,3
46,send_loop.sol,denial_of_service,True,None,None,True,True,3
41,auction.sol,denial_of_service,True,None,True,True,True,4
45,list_dos.sol,denial_of_service,None,True,True,True,True,4


In [410]:
df_sorted[df_sorted['Category']=='front_running']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
47,ERC20.sol,front_running,None,None,None,True,True,2
49,eth_tx_order_dependence_minimal.sol,front_running,True,None,True,True,True,4
48,FindThisHash.sol,front_running,True,None,True,True,True,4
50,odds_and_evens.sol,front_running,True,True,True,True,True,5


In [411]:
df_sorted[df_sorted['Category']=='other']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
52,name_registrar.sol,other,None,None,None,True,True,2
51,crypto_roulette.sol,other,True,None,True,True,True,4
53,open_address_lottery.sol,other,True,None,True,True,True,4


In [412]:
df_sorted[df_sorted['Category']=='reentrancy']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
84,spank_chain_payment.sol,reentrancy,True,None,None,True,False,2
81,reentrancy_insecure.sol,reentrancy,True,True,None,True,True,4
79,reentrancy_cross_function.sol,reentrancy,True,True,None,True,True,4
76,modifier_reentrancy.sol,reentrancy,True,None,True,True,True,4
59,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,reentrancy,True,None,True,True,True,4
77,reentrance.sol,reentrancy,True,True,True,True,True,5
54,0x01f8c4e3fa3edeb29e514cba738d87ce8c091d3f.sol,reentrancy,True,True,True,True,True,5
78,reentrancy_bonus.sol,reentrancy,True,True,True,True,True,5
74,etherbank.sol,reentrancy,True,True,True,True,True,5
80,reentrancy_dao.sol,reentrancy,True,True,True,True,True,5


In [413]:
df_sorted[df_sorted['Category']=='short_addresses']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
85,short_address_example.sol,short_addresses,None,None,None,True,True,2


In [414]:
df_sorted[df_sorted['Category']=='short_addresses']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
85,short_address_example.sol,short_addresses,None,None,None,True,True,2


In [415]:
df_sorted[df_sorted['Category']=='time_manipulation']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
90,timed_crowdsale.sol,time_manipulation,None,None,None,None,True,1
89,roulette.sol,time_manipulation,True,None,True,True,True,4
86,ether_lotto.sol,time_manipulation,True,None,True,True,True,4
87,governmental_survey.sol,time_manipulation,True,True,None,True,True,4
88,lottopollo.sol,time_manipulation,True,True,True,True,True,5


In [416]:
df_sorted[df_sorted['Category']=='unchecked_low_level_calls']

,Contract,Category,sGuard,sGuardPlus,SmartFix,SolGPT,TIPS,PatchCount
114,0x89c1b3807d4c67df034fffb62f3509561218d30b.sol,unchecked_low_level_calls,True,None,None,True,None,2
130,0xe09b1ab8111c2729a76f16de96bc86a7af837928.sol,unchecked_low_level_calls,None,None,True,True,False,2
107,0x663e4229142a27f00bafb5d087e1e730648314c3.sol,unchecked_low_level_calls,None,None,True,None,True,2
96,0x3a0e9acd953ffc0dd18d63603488846a6b8b2b01.sol,unchecked_low_level_calls,True,None,None,True,True,3
119,0xa46edd6a9a93feec36576ee5048146870ea2c3ae.sol,unchecked_low_level_calls,True,None,None,True,True,3
118,0xa1fceeff3acc57d257b917e30c4df661401d6431.sol,unchecked_low_level_calls,True,None,None,True,True,3
117,0x9d06cbafa865037a01d322d3f4222fa3e04e5488.sol,unchecked_low_level_calls,False,None,True,True,True,3
99,0x4051334adc52057aca763453820cb0e045076ef3.sol,unchecked_low_level_calls,True,None,None,True,True,3
101,0x4b71ad9c1a84b9b643aa54fdd66e2dec96e8b152.sol,unchecked_low_level_calls,True,None,None,True,True,3
102,0x524960d55174d912768678d8c606b4d50b79d7b1.sol,unchecked_low_level_calls,True,None,None,True,True,3
